In [1]:
import numpy as np
import torch
import os
import pandas as pd
from sentence_transformers import SentenceTransformer


# Load a pre-trained model

In [2]:
model =SentenceTransformer('msmarco-MiniLM-L-12-v3')

# Define a list of hotel reviews

In [3]:
hotel_reviews = [
    "This hotel has a great location but the rooms are too small.",
    "The staff at this hotel is very friendly and helpful.",
    "The WiFi connection at this hotel is very slow and unreliable.",
    "I had a great stay at this hotel. The room was clean and comfortable.",
    "The food at this hotel is delicious but a bit overpriced."
]

# Embed hotel reviews

In [4]:
hotel_reviews_embds=model.encode(hotel_reviews)

In [5]:
import faiss
import time

# Create an index using FAISS

In [6]:
index = faiss.IndexFlatL2(hotel_reviews_embds.shape[1])
index.add(hotel_reviews_embds)
faiss.write_index(index, 'index_hotel_reviews')
index = faiss.read_index('index_hotel_reviews')

In [7]:
def search(query):
    t=time.time()
    query_vector = model.encode([query])
    k = 2
    top_k = index.search(query_vector, k)
    print('totaltime: {}'.format(time.time()-t))
    return [hotel_reviews[_id] for _id in top_k[1].tolist()[0]]

results=search("food")
print(results)

totaltime: 0.027927875518798828
['The food at this hotel is delicious but a bit overpriced.', 'This hotel has a great location but the rooms are too small.']
